In [1]:
import numpy as np
import os

In [2]:
import librosa
import soundfile
import glob

ModuleNotFoundError: No module named 'librosa'

In [14]:
#path = "./cremad/AudioWAV/"

In [15]:
#directory = os.listdir(path)


In [16]:
# directory


In [17]:
# X  = []
# audios = []
# labels = []

In [18]:
# i = 0
# for audio in directory : 
#     emotion = (audio.split('_')[2])
#     audio_loc = path + "/" +  audio
#     x,sr = librosa.load(audio_loc)
#     audios.append(x)
#     labels.append(emotion)
    
#     i +=1 
#     if(i%10==0):
#         print(i,emotion)
    

In [19]:
# labels=np.asarray(labels)
# audios=np.asarray(audios)

In [20]:
# print(labels.shape)
# print(audios.shape)

In [21]:
# np.save('CremaD_audios.npy', audios)
# np.save('CremaD_labels.npy', labels)

In [2]:
audios = np.load('CremaD_audios.npy', allow_pickle=True)
labels = np.load('CremaD_labels.npy',allow_pickle=True)

In [9]:
labels

array(['ANG', 'FEA', 'NEU', ..., 'ANG', 'ANG', 'ANG'], dtype='<U3')

In [5]:
emotion_dict = {
    0 : "angry",
    1 : "disgust",
    2 : "fear",
    3 : "happy",
    4 : "neutral",
    5 : "sad",
    6 : "surprise"
}

In [24]:
def extract_feature(audio, mfcc, chroma, mel):
    X = audio
    sample_rate = 22050
    if chroma:
        stft=np.abs(librosa.stft(X))
        result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [25]:
def load_data(test_size=0.2):
    X_new,Y_new=[],[]
    idx = 0;
    for audio in audios:
#         emotion = emotion_dict[labels[idx]]
#         print(emotion)
        idx +=1
#         if emotion not in observed_emotions:
#             continue

        feature=extract_feature(audio, mfcc=True, chroma=True, mel=True)
        X_new.append(feature)
#         Y_new.append(emotion)
        
    return (X_new,Y_new)

In [26]:
DATA = load_data()

c:\python38\lib\site-packages\librosa\core\pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


In [27]:
X_new = np.array(DATA[0])
Y_new = np.array(labels)

In [28]:
np.save('X_new.npy', X_new)
np.save('Y_new.npy', Y_new)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold, cross_validate

In [30]:
X_new_train, X_new_test, Y_new_train, Y_new_test  = train_test_split(X_new, Y_new,test_size=0.10,random_state=3)


In [31]:
print((X_new_train.shape[0], X_new_test.shape[0]))


(6697, 745)


In [32]:
model=MLPClassifier(alpha=0.01, batch_size=264, epsilon=1e-08,
                    hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=700, 
                    activation='relu',learning_rate_init=0.001, verbose=True, early_stopping=True)


In [33]:
cvresults=cross_validate(model, X_new_train, Y_new_train, cv=5, return_train_score=True)

Iteration 1, loss = 1.79712756
Validation score: 0.264925
Iteration 2, loss = 1.58188920
Validation score: 0.333955
Iteration 3, loss = 1.51127099
Validation score: 0.350746
Iteration 4, loss = 1.47239198
Validation score: 0.339552
Iteration 5, loss = 1.44109767
Validation score: 0.361940
Iteration 6, loss = 1.42292966
Validation score: 0.384328
Iteration 7, loss = 1.40253274
Validation score: 0.375000
Iteration 8, loss = 1.38556734
Validation score: 0.412313
Iteration 9, loss = 1.36904040
Validation score: 0.380597
Iteration 10, loss = 1.36550337
Validation score: 0.412313
Iteration 11, loss = 1.35353085
Validation score: 0.397388
Iteration 12, loss = 1.33620289
Validation score: 0.419776
Iteration 13, loss = 1.31833305
Validation score: 0.423507
Iteration 14, loss = 1.31122038
Validation score: 0.430970
Iteration 15, loss = 1.29444209
Validation score: 0.404851
Iteration 16, loss = 1.28539348
Validation score: 0.430970
Iteration 17, loss = 1.28069280
Validation score: 0.434701
Iterat

In [34]:
cvresults

{'fit_time': array([ 9.6479454 , 10.52921343,  9.858289  ,  7.94861245,  8.03691435]),
 'score_time': array([0.02415109, 0.02395129, 0.01903558, 0.0211978 , 0.02496123]),
 'test_score': array([0.45820896, 0.45      , 0.45929798, 0.45481703, 0.45631068]),
 'train_score': array([0.55404144, 0.55478813, 0.56532288, 0.55076521, 0.55655095])}

In [23]:
# model.fit(X_new_train, Y_new_train)

Iteration 1, loss = 1.69667434
Validation score: 0.361194
Iteration 2, loss = 1.53826884
Validation score: 0.370149
Iteration 3, loss = 1.47582584
Validation score: 0.382090
Iteration 4, loss = 1.45284402
Validation score: 0.417910
Iteration 5, loss = 1.42056447
Validation score: 0.404478
Iteration 6, loss = 1.39973849
Validation score: 0.414925
Iteration 7, loss = 1.37876462
Validation score: 0.416418
Iteration 8, loss = 1.36208678
Validation score: 0.431343
Iteration 9, loss = 1.34923879
Validation score: 0.431343
Iteration 10, loss = 1.34022418
Validation score: 0.425373
Iteration 11, loss = 1.32120705
Validation score: 0.434328
Iteration 12, loss = 1.30700084
Validation score: 0.441791
Iteration 13, loss = 1.30674906
Validation score: 0.417910
Iteration 14, loss = 1.29194333
Validation score: 0.425373
Iteration 15, loss = 1.28922266
Validation score: 0.437313
Iteration 16, loss = 1.27923643
Validation score: 0.438806
Iteration 17, loss = 1.26383781
Validation score: 0.446269
Iterat

MLPClassifier(activation='tanh', alpha=0.01, batch_size=264, beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=700, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=False)

In [24]:
# Y_pred = model.predict(X_new_test)

In [25]:
cm = confusion_matrix(Y_pred,Y_new_test)

In [26]:
cm

array([[86, 15, 18, 22,  2,  0],
       [11, 40, 12, 16, 16, 11],
       [ 6, 12, 43,  9,  4, 10],
       [11, 11, 13, 51, 15,  3],
       [ 6, 30, 16, 30, 44, 30],
       [ 1, 19, 29, 10, 21, 72]])

In [27]:
from sklearn.model_selection import cross_val_score

In [34]:
score

0.5104477611940299